In [1]:
# import the libraries
import pandas as pd
import numpy as np

In [2]:
# Read the file mammography and check the content
mam = pd.read_csv('../input/mammography.csv')
mam.head()

BIRADS Age Shape Margin Density  Severity
0      5  67     3      5       3         1
1      4  43     1      1       ?         1
2      5  58     4      5       3         1
3      4  28     1      1       3         0
4      5  74     1      5       ?         1

In [3]:
# question 1: Calculate how often breast cancer occurs on average in this data set.
mam['Severity'].mean()

0.4630593132154006

In [4]:
# question 2: Calculate how many of the samples are benign and how many are malignant (benign: 516, malign: 445)
mam.groupby('Severity')['BIRADS'].count()

Severity
0    516
1    445
Name: BIRADS, dtype: int64

In [5]:
# question 3: Missing attributes are indicated as “?”. Remove all lines that contain some “?”. 
mam = mam[~mam.isin(['?']).any(axis=1)]
mam.head()

BIRADS Age Shape Margin Density  Severity
0       5  67     3      5       3         1
2       5  58     4      5       3         1
3       4  28     1      1       3         0
8       5  57     1      5       3         1
10      5  76     1      4       3         1

In [6]:
# Count the number of lines
mam.count()

BIRADS      830
Age         830
Shape       830
Margin      830
Density     830
Severity    830
dtype: int64

In [7]:
# question 4: Calculate the percentage of the BI-RADS assessments that were correct, 
# assuming a BI-RADS assessment of 4 or higher is malign. (51 %)
# First add an extra column BIRADS1 --> 1 (if >= 4 else 0)
mam['Correct'] = np.where(((mam['BIRADS'] >= '4') & (mam['Severity'] == 1)) | 
                         ((mam['BIRADS'] <= '3') & (mam['Severity'] == 0)), 1, 0)
mam.head()

BIRADS Age Shape Margin Density  Severity  Correct
0       5  67     3      5       3         1        1
2       5  58     4      5       3         1        1
3       4  28     1      1       3         0        0
8       5  57     1      5       3         1        1
10      5  76     1      4       3         1        1

In [8]:
# question 4: Calculate the percentage of the BI-RADS assessments that were correct, 
# assuming a BI-RADS assessment of 4 or higher is malign. (51 %)
mam.groupby('Correct')['Severity'].count()

Correct
0    405
1    425
Name: Severity, dtype: int64

In [9]:
mam = mam.drop(['Correct','BIRADS'], axis=1)
mam.head()

Age Shape Margin Density  Severity
0   67     3      5       3         1
2   58     4      5       3         1
3   28     1      1       3         0
8   57     1      5       3         1
10  76     1      4       3         1

In [10]:
# question 5: Use random forest classification to determine a model for predicting the severity (benign/malign) 
# of new mammography results. Determine the accuracy of the classifier (+/- 79%)


In [11]:
from sklearn.model_selection import train_test_split
X = mam.drop('Severity',axis=1)
y = mam['Severity']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30)

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=300)
model.fit(X_train, y_train)

y_test2 = model.predict(X_test)

C:\Program Files\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [12]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_test2)

0.7389558232931727

In [13]:
print(X_train.columns)
print(model.feature_importances_)

Index(['Age', 'Shape', 'Margin', 'Density'], dtype='object')
[0.43301949 0.21198125 0.32475865 0.03024061]


In [14]:
# question 6: Which two factors are the most determining for malign breast tumors. (Age and margin)
# we now combine those two collections into a dataframe
pd.DataFrame(model.feature_importances_,columns=['Importance'],index=X_train.columns).sort_values(by='Importance',ascending=False)

Importance
Age        0.433019
Margin     0.324759
Shape      0.211981
Density    0.030241